In [ ]:
%matplotlib inline
import fceulib
from fceulib import VectorBytes
import numpy as np
import time
import matplotlib.pyplot as plt
from fceu_help import *
from math import *

In [ ]:

START = 0x08
RUN = 0x02
JUMP = 0x01

#inputVec = fceulib.readInputs('videos/algar-batman.fm2')
#emu = fceulib.runGame('batman.nes')

#inputVec = fceulib.readInputs('videos/lordtom_tompa-smb3-warpless.fm2')
#emu = fceulib.runGame('smb3.nes')

inputVec = fceulib.readInputs('videos/happylee4-smb-warpless.fm2')
emu = fceulib.runGame('mario.nes')

#inputVec = fceulib.readInputs('videos/Contra by 3D.fm2')
#emu = fceulib.runGame('contra.nes')

start = VectorBytes()
emu.save(start)



In [ ]:
peekevery = 1
emu.load(start)
image_prefix = 'contra'
display = False
img_buffer = VectorBytes()

colorized2id = {}
id2colorized = {}
with open('test.txt','wb') as outfile:
    for timestep, inp in enumerate(inputVec):
        emu.stepFull(inp,0x0)
        if not (timestep % peekevery == 0) or timestep < 270 :
            continue
        print timestep
        if timestep > 1000:
            break

        sprite_list,colorized_sprites = get_all_sprites(emu.fc)
        if display:
            outputImage(emu, '{}_{}'.format(image_prefix,timestep),img_buffer)
            #if len(sprite_list) > 0:
            #    for ii in range(len(sprite_list)):
            #        print sprite_list[ii][0],sprite_list[ii][1]
            #        plt.imshow((colorized_sprites[ii][:,:,:3]*1.0)/255.0,interpolation='nearest')
            #        plt.show()
            if len(sprite_list) > 0:
                sprite_list2 = np.array(sprite_list,dtype='float')
                print sprite_list2
                plt.plot(sprite_list2[:,0]/8.,sprite_list2[:,1]/8.,'rx') 
            nt = pointer_to_numpy(emu.fc.ppu.NTARAM)
            nt = nt[:960].reshape(30,32)
            plt.imshow(nt,interpolation='none')
            plt.show()
        
        for sprite_id,sprite in enumerate(sprite_list):
            if np.sum(colorized_sprites[sprite_id].ravel()) == 0:
                continue
            uniq = tuple(colorized_sprites[sprite_id].ravel())
            if uniq not in colorized2id:
                colorized2id[uniq] = len(colorized2id)
                id2colorized[colorized2id[uniq]] = colorized_sprites[sprite_id]
            outfile.write('{},{},{},{}\n'.format(timestep,colorized2id[uniq],sprite[0],sprite[1]))
        

In [ ]:


#uses normalized pmi as a thresholding mechanism 
# -1 = no co-occurences, 
# 0 = independent 
# 1 = only co-occur
def getFusedPairs(filename,fc,threshold=0.1):
    #Read in data
    with open(filename) as infile:
        data = []
        for line in infile:
            data.append([int(float(d)) for d in line.rstrip().split(',')])
    #get the per timestep data, as well as total counts for each sprite
    timesteps = {}
    pSprite = {}
    for d in data:
        if d[0] not in timesteps:
            timesteps[d[0]] = []
        timesteps[d[0]].append(d[1:])
        if d[1] not in pSprite:
            pSprite[d[1]] = 0
        pSprite[d[1]] += 1
    spr = get_sprite(0,emu.fc)
    height = spr.shape[0]
    #for each timestep look at all the sprite pairs and see how far apart they are,
    #if they are within width and height of each other, add that as a co-occurence
    
    all_sprites = {}
    for timestep in sorted(timesteps):
        dat = timesteps[timestep]
        for ii in range(len(dat)):
            for jj in range(len(dat)):
                if ii != jj:
                    id1 = dat[ii][0]
                    id2 = dat[jj][0]
                    dx = dat[ii][1]-dat[jj][1]
                    dy = dat[ii][2]-dat[jj][2]
                    
                    if abs(dx) > 8 or abs(dy) > height:
                        continue
                    if id1 not in all_sprites:
                        all_sprites[id1] = {}
                    if id2 not in all_sprites[id1]:
                        all_sprites[id1][id2] = 0
                    all_sprites[id1][id2] += 1
    
    display = True
    accepted = set()
    for sprite in all_sprites:
        total = 0
        dat = []
        px = float(pSprite[sprite])/float(len(timesteps))

        for other in all_sprites[sprite]:
            py = float(pSprite[other])/float(len(timesteps))
            pxy = float(all_sprites[sprite][other])/float(len(timesteps))
            dat.append(log(px*py)/log(pxy)-1)
        if display:
            plt.bar(range(len(all_sprites[sprite])), dat)
            plt.show()
        for other,d in zip(all_sprites[sprite],dat):
            if d > threshold:
                s = sprite
                o = other
                if sprite > other:
                    o = sprite
                    s = other
                accepted.add((s,o))
    return accepted
getFusedPairs('test.txt',emu.fc)

In [ ]:
help(fceulib)